Initializing Table

In [1]:
import tabula
import pandas as pd

Functions

In [2]:
#Extracts key words on the pdf
#Returns a data frame of keywords and it's location on the pdf, with page numbers
#Step 1 in table parsing

from pathlib import Path
from typing import Iterable, Any

from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer


def extract_to_df(pages: Any):
    cols = ["Page","Text", "x_1", "y_1", "x_2", "y_2"]
    words = ["Last Raced", "Fractional Times", "Past Performance Running Line Preview", "Fin", "Trainers"]
    text_loc_df = pd.DataFrame(columns = cols)
    
    page_counter = 0
    for page_layout in pages:
        page_counter += 1
        word_counter = 0
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                element_text = element.get_text()
                for word in words:
                    if(element_text.__contains__(word)):
                        loc_nums = []
                        for num in element.bbox:
                            loc_nums.append(num) #Adding binding box numbers
                        text_loc_df.loc[len(text_loc_df.index)] = [page_counter,
                                                                     word,
                                                                     loc_nums[0], 
                                                                     loc_nums[1], 
                                                                     loc_nums[2], 
                                                                     loc_nums[3]]
    return text_loc_df

In [3]:
#Getting pixel location of words per page
#Step 2 in table parsing

def create_text_loc_df(page_num,df):
    text_loc_df = df.loc[df['Page'] == page_num] 

    #Saving only last instance of fin; Lowest y_1

    fin_df = text_loc_df.loc[text_loc_df['Text'] == "Fin"] #df with only Fin
    fin_df = fin_df.sort_values(by=['y_1']) #Sort by y_1 descending
    fin_row = fin_df.iloc[0] #Save 1st row
    text_loc_df = text_loc_df.loc[text_loc_df['Text'] != "Fin"] #Delete Fin rows
    #text_loc_df = pd.concat(text_loc_df,fin_row.to_frame(),ignore_index=True) #Append right Fin row
    text_loc_df.loc[len(text_loc_df) + 1] = fin_row

    #Resort table by index
    text_loc_df = text_loc_df.sort_index()
    return text_loc_df

In [4]:
#Creating table location dataframe
#Converts to inches and then pdf_location for tabula
#Step 3 of table parsing

def table_location_to_df(page_num,text_loc_df):
    df = text_loc_df
    cols = ["Page","Table",'Top','Left','Bottom','Right']
    table_loc_df = pd.DataFrame(columns = cols)
    page = page_num
    #Top Table

    left = 7.92
    right = 559.44

    y_1 = df.loc[df['Text'] == "Last Raced",'y_1'].tolist()[0] #Getting y_1 value
    top = (((729 - y_1) * 11)/792) *72

    y_1 = df.loc[df['Text'] == "Fractional Times",'y_1'].tolist()[0]
    bottom = (((729 - y_1) * 11)/792 + .50) *72

    #Adding Table 1 to df
    table_loc_df.loc[len(table_loc_df.index)] =[page,1,top,left,bottom,right]

    #Bottom Table

    left = 126
    right = 424.08

    y_1 = df.loc[df['Text'].str.contains("Past Performance"),'y_1'].tolist()[0]
    top = (((729 - y_1) * 11)/792) * 72


    y_1 = df.loc[df['Text'] == "Trainers",'y_1'].tolist()[0]
    bottom = ((((729 - y_1) * 11)/792 + .75)) * 72

    #Adding Table 2 to df
    table_loc_df.loc[len(table_loc_df.index)] =[page,2,top,left,bottom,right]
    return table_loc_df


In [5]:
#Gets the number of horses per race on the pdf 
#Helper function for get_table to find bottom bounds

from PyPDF2 import PdfReader
def number_horses(file, #Pdf file
                 page_num): #Page number of pdf
    reader = PdfReader(file)
    page = reader.pages[page_num - 1]
    text = page.extract_text()
    tokenized_text = text.split()
    
    start_count_bool = False
    count = 0
    for i in range(0,len(tokenized_text) - 1):
        if(tokenized_text[i] == "Trainers:"):
            start_count_bool = True
        elif(tokenized_text[i] == "Owners:"):
            start_count_bool = False
        elif(start_count_bool and tokenized_text[i] == '-'):
            count += 1
    return count

In [6]:
#Gets a table from a pdf, using a dataframe of rough locations of the tables on the pdf
#Returns a dataframe of the table
#Step 4 of table parsing


def get_table(table_loc_df, #Dataframe of locations of tables on pdf
              table_num, #Table we are parsing for (1 = top, 2 = bottom)
              page): #Page number
    print("Looking for tables...")
    #Setting variables
    file = "equibaseFile.pdf"  
    num_horses = number_horses(file,page)
    num_target_rows = num_horses * 2
    
    #Initial table scan
    try:
        scan_df = tabula.read_pdf(file, pages = page, area = [test_area])
        table_df = scan_df[0]
        print(table_df)
        col_names = list(table_df.columns.values)
        num_rows = len(table_df.index)
    except:
        print("Couldn't read a table from default area")
        col_names = ["top_not_found"]
        num_rows = 0
        
    #Finding top bound
    
    
    #Setting header value to look for
    if(table_num == 1):
        target_header = 'Last Raced'
    else:
        target_header = 'Pgm'
        

    
    #Loop until top bound is right
    bound_num = 0 #Increased bounds by 10, both positive and negative to find the right headers. Ex: 10, -10, 20, -20, 30...
    while(col_names[0] != target_header or bound_num > 200):        
        top_bound = table_loc_df.iloc[table_num - 1,2]
        top_bound += bound_num
        test_area = [
            top_bound,
            table_loc_df.iloc[table_num - 1,3],
            table_loc_df.iloc[table_num - 1,4],
            table_loc_df.iloc[table_num - 1,5]
        ]
        try:
            scan_df = tabula.read_pdf(file, pages = page, area = [test_area])
            table_df = scan_df[0]
        except:
            print("Couldn't read a table, moving on from {} top_bound".format(top_bound))
            
        col_names = list(table_df.columns.values)

        #Change bound_num
        if(bound_num <= 0): #If number is in negative cycle
            bound_num *= -1
            bound_num += 10
        else:
            bound_num *= -1
    
    if(bound_num > 200):
        print("Error: Couldn't find top bound")
        return
    else:
        print("Found table top bound at {}".format(top_bound))
        table_loc_df.iloc[table_num - 1, 2] = top_bound #Adding new top bound to df
    
    #Finding bottom bound

    bound_num = 0 #Increased bounds by 10, both positive and negative to find the right headers. Ex: 10, -10, 20, -20, 30...
    while(num_rows != num_target_rows or bound_num > 200):
        bottom_bound = table_loc_df.iloc[table_num - 1,2]
        bottom_bound += bound_num
        test_area = [
            table_loc_df.iloc[table_num - 1,2],
            table_loc_df.iloc[table_num - 1,3],
            bottom_bound,
            table_loc_df.iloc[table_num - 1,5]
        ]
        try:
            scan_df = tabula.read_pdf(file, pages = page, area = [test_area])
            table_df = scan_df[0]
        except:
            print("Couldn't read a table, moving on from {} bottom_bound".format(bottom_bound))
            
        num_rows = len(table_df.index)

         #Change bound_num
        if(bound_num <= 0): #If number is in negative cycle
            bound_num *= -1
            bound_num += 10
        else:
            bound_num *= -1
    
    if(bound_num > 200):
        print("Error: Couldn't find bottom bound")
    else:
        table_loc_df.iloc[table_num - 1, 4] = bottom_bound #Adding new bottom bound to df
        print("Found table bottom bound at {}".format(bottom_bound))
        return table_df

In [ ]:
import warnings
warnings.filterwarnings('ignore')

pdf = "equibaseFile.pdf"
num_pages = 11
df_list = []
#Extract words on position
pages = extract_pages("equibaseFile.pdf")
full_text_loc_df = extract_to_df(pages)
for i in range(1,num_pages): #Get both tables for each page
    print("Page num: " ,i)
    text_loc_df = create_text_loc_df(i,full_text_loc_df) #Locations of key text on page
    table_loc_df = table_location_to_df(i,text_loc_df) #Locations of tables on page
    top_table = get_table(table_loc_df,1,i) #Getting top table
    bottom_table = get_table(table_loc_df,2,i) #Getting bottom table
#Add to list of dfs
df_list.append(top_table)
df_list.append(bottom_table)


The PDF <_io.BufferedReader name='equibaseFile.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Page num:  1
Looking for tables...
Couldn't read a table from default area
Found table top bound at 135.24599999999995


Error from tabula-java:
Exception in thread "main" java.lang.UnsupportedOperationException: Can't add an oblique ruling.
	at technology.tabula.Page.addRuling(Page.java:302)
	at technology.tabula.Page.addBorderRulingsTo(Page.java:159)
	at technology.tabula.Page.getArea(Page.java:134)
	at technology.tabula.CommandLineApp.extractFile(CommandLineApp.java:177)
	at technology.tabula.CommandLineApp.extractFileTables(CommandLineApp.java:124)
	at technology.tabula.CommandLineApp.extractTables(CommandLineApp.java:106)
	at technology.tabula.CommandLineApp.main(CommandLineApp.java:76)




Couldn't read a table, moving on from 135.24599999999995 bottom_bound
Found table bottom bound at 135.24599999999995
Looking for tables...
Couldn't read a table from default area
Found table top bound at 404.8310000000001


Error from tabula-java:
Exception in thread "main" java.lang.UnsupportedOperationException: Can't add an oblique ruling.
	at technology.tabula.Page.addRuling(Page.java:302)
	at technology.tabula.Page.addBorderRulingsTo(Page.java:159)
	at technology.tabula.Page.getArea(Page.java:134)
	at technology.tabula.CommandLineApp.extractFile(CommandLineApp.java:177)
	at technology.tabula.CommandLineApp.extractFileTables(CommandLineApp.java:124)
	at technology.tabula.CommandLineApp.extractTables(CommandLineApp.java:106)
	at technology.tabula.CommandLineApp.main(CommandLineApp.java:76)




Couldn't read a table, moving on from 404.8310000000001 bottom_bound
Found table bottom bound at 404.8310000000001
Page num:  2
Looking for tables...
Couldn't read a table from default area
Found table top bound at 125.82999999999993


Error from tabula-java:
Exception in thread "main" java.lang.UnsupportedOperationException: Can't add an oblique ruling.
	at technology.tabula.Page.addRuling(Page.java:302)
	at technology.tabula.Page.addBorderRulingsTo(Page.java:159)
	at technology.tabula.Page.getArea(Page.java:134)
	at technology.tabula.CommandLineApp.extractFile(CommandLineApp.java:177)
	at technology.tabula.CommandLineApp.extractFileTables(CommandLineApp.java:124)
	at technology.tabula.CommandLineApp.extractTables(CommandLineApp.java:106)
	at technology.tabula.CommandLineApp.main(CommandLineApp.java:76)




Couldn't read a table, moving on from 125.82999999999993 bottom_bound
Found table bottom bound at 125.82999999999993
Looking for tables...
Couldn't read a table from default area


In [ ]:
df_list[19]